In [1]:
from rtt_tools import dump_data
from rtt_tools.dump_data import *  # pussy died because of this wildcard import
from rtt_tools.export import *

logger = logging.getLogger(__name__)
coloredlogs.CHROOT_FILES = []
coloredlogs.install(level=logging.INFO, use_chroot=False)

In [ ]:
from rtt_tools.cleaner import Cleaner
cl = Cleaner()
cl.exp_id_low = 284942
cl.main()

In [2]:
loader = dump_data.Loader()
loader.thr_stest_load = 3000
exporter = Exporter(loader)

In [3]:
exp_id_list = list(range(260259, 260259+2000))

In [ ]:
exporter.load({'no_pvals':False, 'only_pval_cnt': True, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
exporter.export(base_path='/tmp', fname='rtt-results-syso-diehgarder-t05-t06-full.json')
print(exporter.fname)


In [ ]:
ex_start = 261759 + 100  # original start: 260259
ex_end = 260259 + 2000
ex_step = 100

for exr in range(ex_start, ex_end, ex_step):
    loader = dump_data.Loader()
    loader.thr_stest_load = 1000
    exporter = Exporter(loader)
    exp_id_list = list(range(exr, exr + ex_step))
    
    exporter.load({'no_pvals':False, 'only_pval_cnt': False, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
    exporter.export(base_path='/var/tmp/xklinec', fname='rtt-results-syso-diehgarder-t05-t06-%s.json' % (exr,))
    print(exporter.fname)
    

In [ ]:
exr = 262265
loader = dump_data.Loader()
loader.thr_stest_load = 1000
exporter = Exporter(loader)
exp_id_list = list(range(exr, 266975+1))

exporter.load({'no_pvals':False, 'only_pval_cnt': False, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
exporter.export(base_path='/var/tmp/xklinec', fname='rtt-results-syso-diehgarder-t07-nist-qrng-%s.json' % (exr,))
print(exporter.fname)

In [ ]:
exr = 271496
loader = dump_data.Loader()
loader.thr_stest_load = 1000
exporter = Exporter(loader)
exp_id_list = list(range(exr, 283464+1))

exporter.load({'no_pvals':False, 'only_pval_cnt': False, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
exporter.export(base_path='/var/tmp/xklinec', fname='rtt-results-syso-diehgarder-t08-aes-nist-%s.json' % (exr,))
print(exporter.fname)

In [ ]:
exr = 283466
loader = dump_data.Loader()
loader.thr_stest_load = 1000
exporter = Exporter(loader)
exp_id_list = list(range(exr, 284515+1))

exporter.load({'no_pvals':False, 'only_pval_cnt': False, 'experiment_ids': exp_id_list, 'is_secmargins':0}) #  'experiments': exp_id_list})
exporter.export(base_path='/var/tmp/xklinec', fname='rtt-results-syso-diehgarder-t10-aes-nist-stdconf-%s.json' % (exr,))
print(exporter.fname)

In [2]:
from rtt_tools.gen.max_rounds import FUNC_DB, FuncDb, FuncInfo

In [24]:
import json
js = json.load(open('/tmp/rtt-results-full4.json'))

In [25]:
#print(js[2829322])
#for rix, rec in enumerate(js):
#    if 'lowmc' in rec['data_type']:
#        print(rec['data_type'])
#     exp_info = loader.break_exp_ph4(rec['data_type'])
#     if not exp_info:
#         print('Could not parse %s' % rec['data_type'])
#         continue

In [26]:
loader = dump_data.Loader()

def exp_info_to_rec(exp_info):
    return collections.OrderedDict([
        ('tp', exp_info.fnc_type),
        ('f', exp_info.fnc_name),
        ('s', exp_info.size),
        ('m', exp_info.meth),
        ('r', exp_info.fnc_round),
        ('e', exp_info.id),
    ])

def name_to_dict(name):  # js[0]['data_type']
    exp_info = loader.break_exp_ph4(name)
    if exp_info is None or exp_info.fnc is None:
        exp_info = loader.break_exp_ph4_mpc(name)
    if exp_info is None or exp_info.fnc is None:
        return None
    return exp_info_to_rec(exp_info)


In [27]:
funtype = {}  # fun -> type
funrounds = collections.defaultdict(lambda: set())  # fun -> set of rounds tested
methsize = {}
testvariant = {}
datatype = {}
fnames = set()

# func_name -> (meth, size) -> round -> test:var:subtest -> [pass, fail, fail]  # 3 seed results
funcres = collections.defaultdict(                   # fncname
    lambda: collections.defaultdict(                 # meth-size
        lambda: collections.defaultdict(             # round
            lambda: collections.defaultdict(         # test-variant-subtest
                lambda: collections.OrderedDict([        
                    ('info', None),
                    ('recidx', None),
                    ('exs', [None]*3),
]))))) 


for rix, rec in enumerate(js):
    data_type = rec['data_type']
    data_type = data_type.replace('stream:cipher', 'stream_cipher')
    exp_info = loader.break_exp_ph4(data_type)
    if exp_info is None or exp_info.fnc is None:
        exp_info = loader.break_exp_ph4_mpc(data_type)
    if not exp_info or not exp_info.fnc:
        print('Could not parse %s' % data_type)
        continue
    #if rix > 1000:
    #    break
        
    funtype[exp_info.fnc_name] = exp_info.fnc_type
    funrounds[exp_info.fnc_name].add(exp_info.fnc_round)

    meth_size = '%s:%s' % (exp_info.meth, exp_info.size)
    if 'testmpc' in data_type:
        meth_size = '%s:%s:%s' % (exp_info.meth, exp_info.spread, exp_info.size)
    
    methsize[meth_size] = [exp_info.meth, exp_info.size]
    cround = exp_info.fnc_round  # TODO: not always int
    datatype[rec['data_type']] = exp_info_to_rec(exp_info)
    
    test_desc = f'{rec["test"]}##[{rec["variant_type"]}][{rec["variant"]}]##[{rec["subtest_type"]}][{rec["subtest"]}]'
    testvariant[test_desc] = collections.OrderedDict([        
        ('variant_type', rec["variant_type"]),
        ('variant', rec["variant"]),
        ('variant_id', rec["variant_id"]),
        ('subtest_type', rec["subtest_type"]),
        ('subtest', rec["subtest"]),
        ('exid', rec["exid"]),
    ])
    
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['recidx'] = rix
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['info'] = data_type
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['einfo'] = exp_info
    
    # iterate over subtests
    nsubs_passed = 0
    for sub in rec['subs']:
        is_passed = sub['stats'][0]['pass']
        nsubs_passed += int(is_passed)
        
    nsubs = len(rec['subs'])
    npass_ratio = nsubs_passed / nsubs
    variant_passed = npass_ratio >= 0.99  # tune this ratio! Ratio of passed subtests to consider test passed
    
    if exp_info.id >= 3:
        print(f'!! Out of range {exp_info.id} for {data_type}')
        continue
        
    funcres[exp_info.fnc_name][meth_size][cround][test_desc]['exs'][exp_info.id] = variant_passed
    # print('#: %s, pass: %s, pct: %2.7f' % (nsubs, nsubs_passed, 100*nsubs_passed/nsubs))
    
#print(funrounds, funtype, funcres)
#print(json.dumps(funcres, indent=2))
print('Proc1 done')

Proc1 done


In [28]:
# funcres["AES"]["ctr:10485760"][3]
# Analyze function -> meth-size -> round; all tests, reject if 2/3 rejected

# func_name -> (meth, size) -> round -> [pass, fail, fail]
funcres_agg = collections.defaultdict(               # fncname
    lambda: collections.defaultdict(                 # meth-size
        lambda: collections.defaultdict(             # round        
            lambda: collections.OrderedDict([        
                ('num_tests', 0),
                ('num_rej', 0),
                ('rejects', False),
                ('info', None)
])))) 


uncomps = 0
uncomps_funcs = collections.Counter()
uncomps_set = set()
uncomps_bat = set()
rerun_cmds = collections.defaultdict(lambda: set())

bat_map = {
    'NIST Statistical Testing Suite': '--nist_sts',
    'Dieharder': '--dieharder',
    'TestU01 Alphabit': '--tu01_alphabit',
    'TestU01 Small Crush': '--tu01_crush',
    'TestU01 Rabbit': '--tu01_rabbit',
    'TestU01 Block Alphabit': '--tu01_blockalphabit',
    'booltest_1': '--booltest-1',
    'booltest_2': '--booltest-2',
}

expfiles = []
if os.path.exists('expfiles.json'):
    with open('expfiles.json') as fh:
        expfiles = json.load(fh)
        
        
def find_expfile(tpl, expfiles):
    for fname in expfiles:
        if tpl in fname:
            yield fname

            
for fname in funcres:
    for meth_size in funcres[fname]:
        for cround in funcres[fname][meth_size]:
            ctests = funcres[fname][meth_size][cround]  # all tests map
            crec = funcres_agg[fname][meth_size][cround]  # resulting dict
            
            for ctest in ctests:
                for resix, resex in enumerate(ctests[ctest]['exs']):
                    if resex is None:
                        uncomps += 1
                        uncomps_funcs[fname] += 1
                        ename = ctests[ctest]["info"]
                        #if 'testmpc' in ename: 
                        #    continue
                        if '-e:' in ename:
                            ename2 = re.sub(r'-e:[\d]-', f'-e:{resix}-', ename)
                            ename2 = re.sub(r'-i:([\w]+?)-.*', '-i:\\1', ename2)
                            ename2 = re.sub(r'^PH4-SM-[\d]+-', '', ename2)
                            fnames = list(find_expfile('-' + ename2.replace(':', '_'), expfiles))
                            if len(fnames) != 1:
                                print(f'! bad fname count: {fnames}')
                                continue
                            ename = fnames[0]
                            ename = ename.replace('.json', '')
                            ename = ename.replace('_', ':')

                        elif 'testmpc' in ename:
                            ename = re.sub(r'-inp-([\w]+?)([\d]+)-', '-inp-\\1!XXXX', ename)
                            ename = ename.replace('!XXXX', '%02d-' % resix)
                            ename = ename.replace('.json', '')
                            ename = re.sub(r'^.*-testmpc', 'testmpc', ename)
                        
                        uncomps_set.add(ename)
                        uncomps_bat.add(ctest.split('|')[0])
                        rerun_cmds[ename].add(ctest.split('|')[0])
                        print(f'Uncomputed: {fname}-{meth_size}-{cround}-{ctest}-{ctests[ctest]["exs"]}-{ctests[ctest]["recidx"]}, {ename}')
                    
                if None in ctests[ctest]['exs']:                    
                    # print(json.dumps(js[ctests[ctest]["recidx"]], indent=2))
                    continue
                    
                is_rejected = sum(ctests[ctest]['exs']) <= 1  # counting passes, 1 or less passes -> rejected (2+ rejections)
                crec['num_tests'] += 1  # better, number of seed inputs, 3 max at the moment
                crec['num_rej'] += int(is_rejected)
                crec['info'] = ctests[ctest]['info']
                crec['einfo'] = ctests[ctest]['einfo']
            
            if crec['num_tests'] == 0:
                print(f'! zero tests: {fname}-{meth_size}-{cround}')
                continue
                
            cratio = crec['num_rej'] / crec['num_tests']
            crec['rejects'] = cratio >= 0.01  # TUNE THIS! 
            
print('\n--------------------\nUncomputed total:', uncomps, 'functions:', uncomps_funcs, ' batteries:', uncomps_bat)
for ename in sorted(list(rerun_cmds.keys())):
    fname = ename.replace(':', '_')
    fname += '.json'
    bats = sorted(list(rerun_cmds[ename]))
    ename2 = ename.replace('-SM-01-', '-SM-04-')
    ename2 = ename2.replace('-SM-02-', '-SM-04-')
    fsize = ('100' if '100MiB' in fname else '10') if 'MiB' in fname else ('100' if '100MB' in fname else '10')
    print(f"submit_experiment {' '.join([bat_map[x] for x in bats])} --name '{ename2}' --cfg '/home/debian/rtt-home/RTTWebInterface/media/predefined_configurations/{fsize}MB.json' --cryptostreams-config '{fname}'")
    
    

Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Block Alphabit|smultin_MultinomialBitsOver##[Bit NB|Bit R|Bit S|Bit W|Repetitions][100000000|0|32|1|1]##[L|N|Sparse|n|r|s|w][2|1|FALSE|100000000|0|32|1]-[True, None, None]-67288, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Block Alphabit|smultin_MultinomialBitsOver##[Bit NB|Bit R|Bit S|Bit W|Repetitions][100000000|0|32|1|1]##[L|N|Sparse|n|r|s|w][2|1|FALSE|100000000|0|32|1]-[True, None, None]-67288, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Block Alphabit|smultin_MultinomialBitsOver##[Bit NB|Bit R|Bit S|Bit W|Repetitions][100000000|0|32|2|1]##[L|N|Sparse|n|r|s|w][2|1|FALSE|100000000|0|32|2]-[True, None, None]-67289, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-TestU01 Block Alphabit|smultin_MultinomialBitsOver##[Bit NB|Bit R|Bit S|Bit W|Repetitions][1000000

Uncomputed: lowmc-s128d-ctr::104857600-1-booltest_2|halving 128-3-2##[Cfg][128-3-2]##[][]-[False, None, None]-68772, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-booltest_2|halving 384-1-3##[Cfg][384-1-3]##[][]-[False, None, None]-68787, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-booltest_2|halving 384-1-3##[Cfg][384-1-3]##[][]-[False, None, None]-68787, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-booltest_2|halving 512-2-3##[Cfg][512-2-3]##[][]-[False, None, None]-68784, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-booltest_2|halving 512-2-3##[Cfg][512-2-3]##[][]-[False, None, None]-68784, testmpc01-lowmc-s128d-bin-raw-r1-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-1-booltest_2|halving 384-1-1##[Cfg][384-1-1]##[][]-[True, None, None]-68779, test

Uncomputed: lowmc-s128d-ctr::104857600-2-Dieharder|STS Runs Test##[P-sample count][100]##[][]-[True, None, None]-153737, testmpc01-lowmc-s128d-bin-raw-r2-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-2-Dieharder|STS Runs Test##[P-sample count][100]##[][]-[True, None, None]-153737, testmpc01-lowmc-s128d-bin-raw-r2-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-2-Dieharder|STS Serial Test (Generalized)##[P-sample count][100]##[][]-[False, None, None]-153738, testmpc01-lowmc-s128d-bin-raw-r2-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-2-Dieharder|STS Serial Test (Generalized)##[P-sample count][100]##[][]-[False, None, None]-153738, testmpc01-lowmc-s128d-bin-raw-r2-inp-ctr02-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857600-2-Dieharder|RGB Bit Distribution Test##[N-tuple setting|P-sample count][1|100]##[][]-[False, None, None]-153739, testmpc01-lowmc-s128d-bin-raw-r2-inp-ctr01-b16-spr--s100MB
Uncomputed: lowmc-s128d-ctr::104857

Uncomputed: lowmc-s128d-lhw::104857600-2-Dieharder|RGB Bit Distribution Test##[N-tuple setting|P-sample count][3|41]##[][]-[True, None, None]-130839, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-Dieharder|RGB Bit Distribution Test##[N-tuple setting|P-sample count][4|31]##[][]-[True, None, None]-130840, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw01-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-Dieharder|RGB Bit Distribution Test##[N-tuple setting|P-sample count][4|31]##[][]-[True, None, None]-130840, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-Dieharder|RGB Bit Distribution Test##[N-tuple setting|P-sample count][5|24]##[][]-[True, None, None]-130841, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw01-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-Dieharder|RGB Bit Distribution Test##[N-tuple setting|P-sample count][5|24]##[][]-[True, None, None]-130841, te

Uncomputed: lowmc-s128d-lhw::104857600-2-NIST Statistical Testing Suite|Test for the Longest Run of Ones in a Block##[Stream count|Stream size][800|1000000]##[][]-[True, None, None]-130476, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw01-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-NIST Statistical Testing Suite|Test for the Longest Run of Ones in a Block##[Stream count|Stream size][800|1000000]##[][]-[True, None, None]-130476, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-NIST Statistical Testing Suite|Random Binary Matrix Rank Test##[Stream count|Stream size][800|1000000]##[][]-[True, None, None]-130477, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw01-b16-w5-spr--s100MB
Uncomputed: lowmc-s128d-lhw::104857600-2-NIST Statistical Testing Suite|Random Binary Matrix Rank Test##[Stream count|Stream size][800|1000000]##[][]-[True, None, None]-130477, testmpc01-lowmc-s128d-bin-raw-r2-inp-lhw02-b16-w5-spr--s100MB
Uncomputed: lowmc-

...

Uncomputed: LBLOCK-ctr:10485760-5-NIST Statistical Testing Suite|Random Excursions Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, None]-2828874, PH4-SM-01-LBLOCK-t:block-r:5-b:8-s:10MiB-e:0-i:ctr-64sbit-offset-0
Uncomputed: LBLOCK-ctr:10485760-5-NIST Statistical Testing Suite|Random Excursions Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, None]-2828874, PH4-SM-01-LBLOCK-t:block-r:5-b:8-s:10MiB-e:2-i:ctr-64sbit-offset-2
Uncomputed: LBLOCK-ctr:10485760-5-NIST Statistical Testing Suite|Random Excursions Variant Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, None]-2828875, PH4-SM-01-LBLOCK-t:block-r:5-b:8-s:10MiB-e:0-i:ctr-64sbit-offset-0
Uncomputed: LBLOCK-ctr:10485760-5-NIST Statistical Testing Suite|Random Excursions Variant Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, None]-2828875, PH4-SM-01-LBLOCK-t:block-r:5-b:8-s:10MiB-e:2-i:ctr-64sbit-offset-2
Uncomputed: LBLOCK-ctr:10485760-4-NIST Statistical Testing Suite

Uncomputed: RIPEMD160-hw:10485760-6-NIST Statistical Testing Suite|Random Excursions Test##[Stream count|Stream size][80|1000000]##[][]-[False, None, None]-2828343, PH4-SM-01-RIPEMD160-t:hash-r:6-b:20-s:10MiB-e:1-i:hw-160bit-hw4-offsetidx-8764786-offset-15-30-110-142-r0.33-vecsize-17529574-10MiB
Uncomputed: RIPEMD160-hw:10485760-6-NIST Statistical Testing Suite|Random Excursions Test##[Stream count|Stream size][80|1000000]##[][]-[False, None, None]-2828343, PH4-SM-01-RIPEMD160-t:hash-r:6-b:20-s:10MiB-e:2-i:hw-160bit-hw4-offsetidx-17529573-offset-38-41-89-114-r0.67-vecsize-8764787-10MiB
Uncomputed: RIPEMD160-hw:10485760-6-NIST Statistical Testing Suite|Random Excursions Variant Test##[Stream count|Stream size][80|1000000]##[][]-[False, None, None]-2828344, PH4-SM-01-RIPEMD160-t:hash-r:6-b:20-s:10MiB-e:1-i:hw-160bit-hw4-offsetidx-8764786-offset-15-30-110-142-r0.33-vecsize-17529574-10MiB
Uncomputed: RIPEMD160-hw:10485760-6-NIST Statistical Testing Suite|Random Excursions Variant Test##[St

Uncomputed: F-FCSR-ctr:104857600-1-TestU01 Small Crush|smarsa_BirthdaySpacings##[Repetitions][1]##[N|d|n|p|r|t][1|1073741824|5000000|1|0|2]-[None, False, False]-2827419, PH4-SM-01-F-FCSR-t:stream:cipher-r:1-b:16-s:100MiB-e:0-i:ctr-128sbit-offset-0
Uncomputed: F-FCSR-ctr:104857600-1-TestU01 Small Crush|sknuth_Collision##[Repetitions][1]##[N|Sparse|d|n|r|t][1|TRUE|65536|5000000|0|2]-[None, False, False]-2827420, PH4-SM-01-F-FCSR-t:stream:cipher-r:1-b:16-s:100MiB-e:0-i:ctr-128sbit-offset-0
Uncomputed: F-FCSR-ctr:104857600-1-TestU01 Small Crush|sknuth_CouponCollector##[Repetitions][1]##[N|d|n|r][1|16|500000|26]-[None, True, False]-2827421, PH4-SM-01-F-FCSR-t:stream:cipher-r:1-b:16-s:100MiB-e:0-i:ctr-128sbit-offset-0
Uncomputed: F-FCSR-ctr:104857600-1-TestU01 Small Crush|sknuth_MaxOft##[Repetitions][1]##[N|d|n|r|t][1|100000|2000000|0|6]-[None, True, True]-2827422, PH4-SM-01-F-FCSR-t:stream:cipher-r:1-b:16-s:100MiB-e:0-i:ctr-128sbit-offset-0
Uncomputed: F-FCSR-ctr:104857600-1-TestU01 Small C

Uncomputed: PICCOLO-ctr:10485760-2-NIST Statistical Testing Suite|Random Excursions Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, None]-2828559, PH4-SM-01-PICCOLO-t:block-r:2-b:8-s:10MiB-e:0-i:ctr-64sbit-offset-0
Uncomputed: PICCOLO-ctr:10485760-2-NIST Statistical Testing Suite|Random Excursions Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, None]-2828559, PH4-SM-01-PICCOLO-t:block-r:2-b:8-s:10MiB-e:2-i:ctr-64sbit-offset-2
Uncomputed: PICCOLO-ctr:10485760-2-NIST Statistical Testing Suite|Random Excursions Variant Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, None]-2828560, PH4-SM-01-PICCOLO-t:block-r:2-b:8-s:10MiB-e:0-i:ctr-64sbit-offset-0
Uncomputed: PICCOLO-ctr:10485760-2-NIST Statistical Testing Suite|Random Excursions Variant Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, None]-2828560, PH4-SM-01-PICCOLO-t:block-r:2-b:8-s:10MiB-e:2-i:ctr-64sbit-offset-2
Uncomputed: PICCOLO-hw:10485760-1-NIST Statistical Testi

Uncomputed: GOST-hw:10485760-5-NIST Statistical Testing Suite|Random Excursions Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, False]-2828814, PH4-SM-01-GOST-t:block-r:5-b:8-s:10MiB-e:0-i:hw-64bit-hw5-offsetidx-0-offset-0-1-2-3-4-r0.00-vecsize-7624512-10MiB
Uncomputed: GOST-hw:10485760-5-NIST Statistical Testing Suite|Random Excursions Variant Test##[Stream count|Stream size][80|1000000]##[][]-[None, False, False]-2828815, PH4-SM-01-GOST-t:block-r:5-b:8-s:10MiB-e:0-i:hw-64bit-hw5-offsetidx-0-offset-0-1-2-3-4-r0.00-vecsize-7624512-10MiB
Uncomputed: ROAD-RUNNER-K128-hw:104857600-1-NIST Statistical Testing Suite|Random Excursions Test##[Stream count|Stream size][800|1000000]##[][]-[None, False, False]-2828769, PH4-SM-01-ROAD-RUNNER-K128-t:block-r:1-b:8-s:100MiB-e:0-i:hw-64bit-hw6-offsetidx-0-offset-0-1-2-3-4-5-r0.00-vecsize-74974368-100MiB
Uncomputed: ROAD-RUNNER-K128-hw:104857600-1-NIST Statistical Testing Suite|Random Excursions Variant Test##[Stream count|Stream size]

In [31]:
log_lines = []
table_lines = []

def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
    return [int(text) if text.isdigit() else text.lower()
            for text in _nsre.split(s)]


for fname in sorted(funcres_agg.keys(), key=natural_sort_key):
    fname_low = fname.lower()
    
    # Find info per-fname
    fmeth = list(funcres_agg[fname].keys())
    fround = list(funcres_agg[fname][fmeth[0]].keys()) if fmeth else None
    finfo = funcres_agg[fname][fmeth[0]][fround[0]] if fround else None
    finfo = finfo['einfo'] if finfo and 'einfo' in finfo else None
        
    erec = FUNC_DB.search(fname, FuncInfo.HASH if finfo is not None and finfo.fnc_type == 'hash' else FuncInfo.CIPHER)
    if erec is None and finfo.fnc_type == 'prng':
        erec = FuncInfo(fname, FuncInfo.PRNG, 0, 1, 1)
    
    max_rounds_cur = erec.max_rounds if erec else None
    crypt_rounds_cur = erec.human_broken_rounds if erec else None
    max_rounds_cur_s = ('%3d' % max_rounds_cur) if max_rounds_cur is not None else ' - '
    crypt_rounds_cur_s = ('%3d' % crypt_rounds_cur) if crypt_rounds_cur is not None else ' - '
    max_rnd_broken = None
    
    for meth in sorted(funcres_agg[fname].keys()):
        namemeth = funcres_agg[fname][meth]
        if '104857600' not in meth: # or 'sac:' in meth:
            continue
        
        rounds = list(namemeth.keys())
        for ix, rnd in enumerate(rounds):
            if rnd == '1-0-0':
                rounds[ix] = 1
                namemeth[1] = namemeth['1-0-0']
            if rnd == 'x':
                namemeth[1] = namemeth['x']
                rounds[ix] = 1
            if rnd is None:
                namemeth[1] = namemeth[None]
                rounds[ix] = 1
                
        rounds.sort()
        for rnd in rounds:
            crec = namemeth[rnd]
            if crec['rejects'] and (max_rnd_broken is None or rnd > max_rnd_broken):
                max_rnd_broken = rnd
            
            margin_s = ('margin: %.2f %%' % (100.0 * (1 - rnd/max_rounds_cur))) if crec['rejects'] and max_rounds_cur is not None else '' 
            vdict = '%s (%3d/%3d | %s %s max. %s)' % ('R' if crec['rejects'] else '-', crec["num_rej"], crec["num_tests"], crypt_rounds_cur_s, max_rounds_cur_s, margin_s)
            irnd = rnd
            try:
                irnd = int(rnd)
            except:
                pass
            srnd = ('%4d' if isinstance(irnd, int) else '%4s') % irnd
            
            log_lines.append('%30s | %20s | %s | %s' % (fname, meth, srnd, vdict))
        
        
    if max_rnd_broken is None and crypt_rounds_cur is None and max_rounds_cur is None:
        print('Skipping null %s' % fname)
        continue
    
    nfname = fname
    nfname = nfname.replace('ROAD-RUNNER-K80', 'R.RUNNER.K80')
    nfname = nfname.replace('ROAD-RUNNER-K128', 'R.RUNNER.K128')
    nfname = nfname.replace('RECTANGLE-K80', 'RECT.K80')
    nfname = nfname.replace('RECTANGLE-K128', 'RECT.K128')
    
    nfname = nfname.replace('testu01-uxorshift', 'U01.XorShift')
    nfname = nfname.replace('testu01-umrg', 'U01.UMRG')
    nfname = nfname.replace('testu01-ulcg', 'U01.ULCG')
    nfname = nfname.replace('std_subtract_with_carry', 'STD.SubCarry')
    nfname = nfname.replace('std_mersenne_twister', 'STD.MTwister')
    nfname = nfname.replace('std_lcg', 'STD.LCG')
    
    nfname = nfname.replace('mimc_hash-', 'MiMC.')
    nfname = nfname.replace('gmimc-', 'GMiMC.')
    nfname = nfname.replace('lowmc-s', 'LowMC.S')
    nfname = nfname.replace('Vision_', 'Vision.')
    nfname = nfname.replace('Starkad_', 'Starkad.')
    nfname = nfname.replace('Rescue_', 'Rescue.')
    nfname = nfname.replace('Poseidon_S128_BLS12_138', 'Poseidon.BLS12')
    nfname = nfname.replace('Poseidon_', 'Poseidon.')
    
    cline = '%s & \\boxSecMarginXX{%s}{%s}{%s}' % (nfname, max_rnd_broken or 0, max_rounds_cur or 1, crypt_rounds_cur or 0)
    table_lines.append((cline, erec))

    
print("\n".join(log_lines))

cols = 3
per_col = math.ceil(len(table_lines)/3.0)
sorter = lambda x: (x[1].ftype, x[1].stype, natural_sort_key(x[1].fname))
table_lines.sort(key=sorter)
ntable_lines = []

print("\n\nTable lines: ", len(table_lines), ", per col:", per_col)
# for k, g in itertools.groupby(table_lines, lambda x: (x[1].ftype)):

old_type=None
for e in table_lines:
    if (e[1].ftype, e[1].stype) != old_type:
        old_type = (e[1].ftype, e[1].stype)
        if old_type[0] == FuncInfo.HASH:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Hash functions}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 0:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Block ciphers}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 2:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{Stream ciphers}}')
        elif old_type[0] == FuncInfo.CIPHER and old_type[1] == 4:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{MPC ciphers}}')
        elif old_type[0] == FuncInfo.PRNG:
            ntable_lines.append('\\multicolumn{3}{|l|}{\\textbf{PRNGs}}')
        
    ntable_lines.append(e[0])

print(len(ntable_lines))
per_col = math.ceil(len(ntable_lines)/3.0)
arcols = [ntable_lines[i*per_col: (i+1)*per_col] for i in range(cols)]

for x in range(per_col):
    tup = arcols[0][x] if x < len(arcols[0]) else '& ', arcols[1][x] if x < len(arcols[1]) else '& ', arcols[2][x] if x < len(arcols[2]) else '& '
    print("& ".join(tup) + "\\\\")

#for x in range(per_col):
#    l1 =    
#print("\n".join(ntable_lines))
   
            

                        Abacus |        ctr:104857600 |    0 | - (  0/245 |  -  280 max. )
                        Abacus |        ctr:104857600 |    1 | - (  0/245 |  -  280 max. )
                        Abacus |        ctr:104857600 |    2 | - (  0/245 |  -  280 max. )
                        Abacus |        ctr:104857600 |    3 | - (  0/245 |  -  280 max. )
                        Abacus |         hw:104857600 |    0 | - (  0/245 |  -  280 max. )
                        Abacus |         hw:104857600 |    1 | - (  0/245 |  -  280 max. )
                        Abacus |         hw:104857600 |    2 | - (  0/245 |  -  280 max. )
                        Abacus |         hw:104857600 |    3 | - (  0/245 |  -  280 max. )
                        Abacus |        sac:104857600 |    0 | - (  0/245 |  -  280 max. )
                        Abacus |        sac:104857600 |    1 | - (  0/245 |  -  280 max. )
                        Abacus |        sac:104857600 |    2 | - (  0/245 |  -  280 max. )